## 2.0 Learning Objectives

By the end of this lab, you would have learnt the following:

1.   How to use google colab
2.   The difference between deterministic profilers and statistical profilers.
3.   How to profile the **running time** of a piece of python code.
4.   How to profile the **memory consumption** of a piece of python code.
5.   How to profile the **cpu usage** of a piece of python code.

## 2.1.1 Google Colab

The Google colab is a free cloud service which allows us to execute code on Google's cloud servers. It is extremely convenient for us as it provides all of us with the same environment to work off from. Let's see what hardware Google is providing us with:

In [1]:
from psutil import *
cpu_count()

20

Typically you will see the value of two here. As of now, google provides us with a dual core Intel Xeon either 2.20 or 2.30 GHz. You will be able to get more details of the software / hardware from the following:


## 2.2 Profiler types

In this section of the lab, we will be working in particular with profilers in the Python language. In general, there are at least two types of profilers in python. They are deterministic and statistical profiling. The differences are as follows:

**Deterministic profiling**

Deterministic profilers monitors all function calls, function returns and exception events. Timings are made for every single one of this events.

**Statistical profiling**

Instead of tracking every event (e.g. call to every function), statistical profilers interrupts the application periodically and samples the effective instruction pointer. These are analyzed next to estimate where the application is spending its time in.

*source: https://docs.python.org/2.0/lib/Deterministic_Profiling.html*

Typically deterministic profilers cause huge overheads as every single event is instrumented. However naturally it provides high granularity and accuracy of the data generated.

## 2.3 Profiling Time

We will be using two functions to measure time. The first is via the time library, using either the perf_counter module or just simply using the default_timer() function to get the current time. To illustrate this, let's use our favourite sorting algorithm, insertion sort.

In [2]:
def insertion_sort(arr):
  for i in range(1, len(arr)):
    curr_val = arr[i]
    curr_index = i
    while curr_index > 0 and arr[curr_index-1] > curr_val:
      arr[curr_index] = arr[curr_index-1]
      curr_index -= 1

    arr[curr_index] = curr_val

In [3]:
import random

array = []
def init_random_array(array_size):
  global array

  array = []
  random.seed(3102)

  for i in range(array_size):
      array.append(random.randint(0, array_size))
  return (array)

### 2.3.1 time.perf_counter

Let's find out how long insertion sort takes on these numbers using time.perf_counter.

In [4]:
from time import perf_counter

init_random_array(1000)

start = perf_counter()
insertion_sort(array)
duration = perf_counter() - start
print(duration)
print(array[:10])

0.032610900001600385
[0, 1, 1, 2, 3, 3, 4, 8, 9, 10]


### 2.3.2 timeit

Another way (and some may argue a better way) is to use the timeit function. This allows us to measure how long a function takes to execute, when it is running many iterations.

In [5]:
from timeit import timeit

init_random_array(1000)
timeit("insertion_sort(array)", 'from __main__ import insertion_sort, array', number=1)


0.04126689999247901

Let's now compare insertion sort with merge sort.

In [6]:
def merge_sort(array, left_index, right_index):
    if left_index >= right_index:
        return

    middle = (left_index + right_index)//2
    merge_sort(array, left_index, middle)
    merge_sort(array, middle + 1, right_index)
    merge(array, left_index, right_index, middle)

def merge(array, left_index, right_index, middle):
    # Make copies of both arrays we're trying to merge

    # The second parameter is non-inclusive, so we have to increase by 1
    left_copy = array[left_index:middle + 1]
    right_copy = array[middle+1:right_index+1]

    # Initial values for variables that we use to keep
    # track of where we are in each array
    left_copy_index = 0
    right_copy_index = 0
    sorted_index = left_index

    # Go through both copies until we run out of elements in one
    while left_copy_index < len(left_copy) and right_copy_index < len(right_copy):

        # If our left_copy has the smaller element, put it in the sorted
        # part and then move forward in left_copy (by increasing the pointer)
        if left_copy[left_copy_index] <= right_copy[right_copy_index]:
            array[sorted_index] = left_copy[left_copy_index]
            left_copy_index = left_copy_index + 1
        # Opposite from above
        else:
            array[sorted_index] = right_copy[right_copy_index]
            right_copy_index = right_copy_index + 1

        # Regardless of where we got our element from
        # move forward in the sorted part
        sorted_index = sorted_index + 1

    # We ran out of elements either in left_copy or right_copy
    # so we will go through the remaining elements and add them
    while left_copy_index < len(left_copy):
        array[sorted_index] = left_copy[left_copy_index]
        left_copy_index = left_copy_index + 1
        sorted_index = sorted_index + 1

    while right_copy_index < len(right_copy):
        array[sorted_index] = right_copy[right_copy_index]
        right_copy_index = right_copy_index + 1
        sorted_index = sorted_index + 1

# credits: https://stackabuse.com/merge-sort-in-python/

In [7]:
init_random_array(1000)
timeit("merge_sort(array, 0, len(array) - 1)", 'from __main__ import merge_sort, array', number=1)

0.0033961000008275732

### 2.3.3 cProfile

Let's first look at a simple iterative example. Assume that we have 5 functions, and they are related to each other in the following manner:

```
func_a -> func_b (1000 times) -> func_e (17000 times)
       -> func_c (1000 times) -> func_e (15000 times)
```

In [8]:
import timeit

tic = timeit.default_timer();

def func_a():
  for i in range(0,1000):
    func_b()
    func_c()

def func_b():
  for i in range(0,17000):
    func_e()

def func_c():
  for i in range(0,15000):
    func_e()

def func_e():
  return

func_a()
toc = timeit.default_timer();
print(f"{toc-tic}")

1.2257314000162296


If we were to run the code above, the time taken is within the range of 2 to 3 seconds. Let's now try to profile the code with the cProfile module. Using the magic %% operator, we are able to write the cell contents to the file in our google cloud server.

In [9]:
%%writefile multiple_calls.py

def func_a():
  for i in range(0,1000):
    func_b()
    func_c()

def func_b():
  for i in range(0,17000):
    func_e()

def func_c():
  for i in range(0,15000):
    func_e()

def func_e():
  return

func_a()

Writing multiple_calls.py


And now the functions can be profiled with the following line:

In [11]:
!python -m cProfile multiple_calls.py

         32002004 function calls in 6.544 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    6.544    6.544 {built-in method builtins.exec}
        1    0.000    0.000    6.544    6.544 multiple_calls.py:1(<module>)
        1    0.002    0.002    6.544    6.544 multiple_calls.py:2(func_a)
     1000    2.367    0.002    3.484    0.003 multiple_calls.py:7(func_b)
     1000    2.083    0.002    3.058    0.003 multiple_calls.py:11(func_c)
 32000000    2.092    0.000    2.092    0.000 multiple_calls.py:15(func_e)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




We can sort it with the -s option.

Notice that the execution time increases almost three times. But here you can see clearly the various calls being made, total time being spent as well as the cummulative time. The column headings are:

**ncalls**
Total number of calls.

**tottime**
Total time spent in the given function (and excluding time made in calls to sub-functions).

**percall**
tottime divided by ncalls

**cumtime**
Cumulative time spent in this and all subfunctions (from invocation till exit).

**percall**
cumtime divided by primitive calls

**filename:lineno(function)**
Line number of the function

*Additional options can be seen here: https://docs.python.org/3.7/library/profile.html*


Let's try to visualize these calls. We first install the necessary packages:

In [12]:
!pip install gprof2dot


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Next we generate the output files in the following manner. Notice that the files output.png and output.pstats are generated on the folder structure on the left.

In [13]:
!python -m cProfile -o output.pstats multiple_calls.py
!gprof2dot -f pstats output.pstats | dot -Tpng -o output.png

'dot' is not recognized as an internal or external command,
operable program or batch file.


The following line displays the output image on your notebook. You can see that it corresponds to the original call tree up above.

In [31]:
from IPython.display import Image
Image(filename='output.pstats')



ValueError: Cannot embed the 'pstats' image format

Now let's try to carry out this analysis on our favourite fibonacci number recursive function, and see if there is a way to optimize it. First we find out what the execution time is when n is equals to 10.

In [16]:
tic = timeit.default_timer();
fib_num = 34
def  fib(n):
    if n == 0: return 0
    if n == 1: return 1
    return fib(n-1) + fib(n-2)

print(f"{fib(fib_num)} {timeit.default_timer() - tic}")

5702887 1.3407636000192724


Let's try to do the profiling here from a module. For this recursive function, it's clearer to use the cProfile as a module rather than from the command line.

In [17]:
import cProfile
import timeit
tic = timeit.default_timer();

fib_num = 34
def  fib(n):
    if n == 0: return 0
    if n == 1: return 1
    return fib(n-1) + fib(n-2)


cProfile.run('print(f\"{fib(fib_num)} {timeit.default_timer() - tic}\")')

5702887 4.330191899993224
         18455201 function calls (267 primitive calls) in 4.331 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
18454929/1    4.329    0.000    4.330    4.330 178620101.py:7(fib)
        1    0.000    0.000    0.001    0.001 <decorator-gen-17>:1(writeout_cache)
        1    0.000    0.000    0.000    0.000 <frozen abc>:121(__subclasscheck__)
        1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1390(_handle_fromlist)
        1    0.000    0.000    0.000    0.000 attrsettr.py:43(__getattr__)
        1    0.000    0.000    0.000    0.000 attrsettr.py:66(_get_attr_opt)
        1    0.000    0.000    0.000    0.000 base_events.py:1903(_timer_handle_cancelled)
        2    0.000    0.000    0.000    0.000 base_events.py:1908(_run_once)
        4    0.000    0.000    0.000    0.000 base_events.py:2003(get_debug)
        1    0.000    0.000    0.000    0.000 base_events.py:446(create

Notice that the execution time has almost doubled with such a small example. You can also tell that the recursive call occurs for more than 18454929 times, although the number of primitive calls are just 40. Hence you can deduce that there is room for improvement here. Let's try to carry out some memoization to reduce the number of function calls.

We do that by first creating a dictionary and storing the outputs there. If the output is present, we will use the value in the dictionary. Else we will carry out the function call.

In [18]:
import cProfile
import timeit

tic = timeit.default_timer();

fib_num = 34

def fib(fib_dict, n):
  if n in fib_dict: return fib_dict[n]

  if n == 0: return 0
  if n == 1: return 1

  if (n-1) in fib_dict:
    return_1 = fib_dict.get(n-1)
  else:
    return_1 = fib(fib_dict, n-1)
    fib_dict[n-1] = return_1

  if (n-2) in fib_dict:
    return_2 = fib_dict.get(n-2)
  else:
    return_2 = fib(fib_dict, n-1)
    fib_dict[n-2] = return_2

  fib_dict[n] = fib_dict[n-1] + fib_dict[n-2]
  return fib_dict[n]

def fib_driver(n):
  fib_dict = {}
  fib(fib_dict, n)
  print(fib_dict[n-1])

cProfile.run('print(f\"{fib_driver(fib_num)} {timeit.default_timer() - tic}\")')

5702887
None 0.0009761999826878309
         127 function calls (93 primitive calls) in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 3309140176.py:29(fib_driver)
     35/1    0.000    0.000    0.000    0.000 3309140176.py:8(fib)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 iostream.py:138(_event_pipe)
        1    0.000    0.000    0.000    0.000 iostream.py:259(schedule)
        4    0.000    0.000    0.000    0.000 iostream.py:505(parent_header)
        4    0.000    0.000    0.000    0.000 iostream.py:550(_is_master_process)
        4    0.000    0.000    0.000    0.000 iostream.py:577(_schedule_flush)
        4    0.000    0.000    0.000    0.000 iostream.py:655(write)
        1    0.000    0.000    0.000    0.000 socket.py:626(send)
        1    0.000    0.000    0.000    0.000 threading.py:1153(_wa

You would notice that the number of iterative calls have decreased dramatically to just 35, which in fact makes this recursive function O(n), the same as the iterative version of our fibonacci number generator.

In [19]:
import cProfile
import timeit

tic = timeit.default_timer();
fib_num = 35
def fib_iter(n):
    fib_0 = 1
    fib_1 = 1
    fib_2 = 0
    for i in range(2,n):
        fib_2 = fib_0 + fib_1
        fib_0 = fib_1
        fib_1 = fib_2
    return fib_2

cProfile.run('print(f\"{fib_iter(fib_num)} {timeit.default_timer() - tic}\")')

9227465 0.00028630002634599805
         37 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 948140518.py:6(fib_iter)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 iostream.py:138(_event_pipe)
        1    0.000    0.000    0.000    0.000 iostream.py:259(schedule)
        2    0.000    0.000    0.000    0.000 iostream.py:505(parent_header)
        2    0.000    0.000    0.000    0.000 iostream.py:550(_is_master_process)
        2    0.000    0.000    0.000    0.000 iostream.py:577(_schedule_flush)
        2    0.000    0.000    0.000    0.000 iostream.py:655(write)
        1    0.000    0.000    0.000    0.000 socket.py:626(send)
        1    0.000    0.000    0.000    0.000 threading.py:1153(_wait_for_tstate_lock)
        1    0.000    0.000    0.000    0.000 threading.py:1220(is_alive)
   

As you can see, it is in the same order as that of the iterative version we have been working with so far.

### 2.3.4 py-spy

We will now compare cProfile with a statistical profiler, py-spy.

In [2]:
!pip install py-spy


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Let's first create a python script which runs for at least 10 seconds.

In [3]:
%%writefile long_multiple_calls.py

def func_a():
  for i in range(0,1000):
    func_b()
    func_c()

def func_b():
  for i in range(0,77000):
    func_e()

def func_c():
  for i in range(0,55000):
    func_e()

def func_e():
  return

func_a()

Overwriting long_multiple_calls.py


What the next line does is to run the function, sampling it at various intervals. In this instance, it is sampled at 100 times per second.

In [5]:
!py-spy record -o profile.svg -- python long_multiple_calls.py


Error: Failed to find python version from target process


In [ ]:
from IPython.core.display import SVG
SVG(filename='profile.svg')

What we see on top is known as a flame graph. We can discuss a bit more on this flame graph.

The flame graph is a collection of stack traces and these traces are arranged as column of boxes, where each box represents a function or a stack frame.

The y-axis shows the depth and in this case, it is ordered from the top (root) to the bottom (leaf).

The x-axis spans the stack trace collection and the ordering is in alphabetical, and not by time.

The width of each function box shows the frequency at which that function was present in the stack traces. So in this example, it shows clearly that the bulk of the cummulative time was spent in calling function_b and function_c from function_a.

The background colors for each box are chosen purely for asthestic purposes and have no inherent meaning.

*source: https://cacm.acm.org/magazines/2016/6/202665-the-flame-graph/fulltext*

One of the useful commands is the Top command. Top shows a live view of what functions are taking the most time in your python program. Running py-spy with the top command gives you the following live statistical updates:

'pyenv' is not recognized as an internal or external command,
operable program or batch file.
'pyenv' is not recognized as an internal or external command,
operable program or batch file.


Python 3.12.2


Let us compare this with cProfile to see if there is any delta between the two.

In [33]:
!python -m cProfile -s tottime long_multiple_calls.py

         132002004 function calls in 20.957 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1000    8.318    0.008   12.223    0.012 long_multiple_calls.py:7(func_b)
132000000    6.694    0.000    6.694    0.000 long_multiple_calls.py:15(func_e)
     1000    5.944    0.006    8.734    0.009 long_multiple_calls.py:11(func_c)
        1    0.000    0.000   20.957   20.957 long_multiple_calls.py:2(func_a)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000   20.957   20.957 {built-in method builtins.exec}
        1    0.000    0.000   20.957   20.957 long_multiple_calls.py:1(<module>)




### 2.4 Profiling Memory

We will be showcasing three tools to profile memory. All three tools have their different strengths / weaknesses, and the choice of the tool is dependent on the situation.

### 2.4.1 Memory Profiler

In [34]:
!pip install memory_profiler


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
%load_ext memory_profiler

In [36]:
%%writefile insert_sort.py

import random
from memory_profiler import profile

array = []
def init_random_array(array_size):
  global array

  array = []
  random.seed(3102)

  for i in range(array_size):
      array.append(random.randint(0, array_size))
  return (array)

@profile
def insertion_sort(arr):
  for i in range(1, len(arr)):
    curr_val = arr[i]
    curr_index = i
    while curr_index > 0 and arr[curr_index-1] > curr_val:
      arr[curr_index] = arr[curr_index-1]
      curr_index -= 1

    arr[curr_index] = curr_val

init_random_array(1000)
insertion_sort(array)

Writing insert_sort.py


In [37]:
!python insert_sort.py

Filename: c:\Users\Vianiece\Desktop\ICT3113\ICT3113\insert_sort.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    16     52.7 MiB     52.7 MiB           1   @profile
    17                                         def insertion_sort(arr):
    18     52.7 MiB      0.0 MiB        1000     for i in range(1, len(arr)):
    19     52.7 MiB      0.0 MiB         999       curr_val = arr[i]
    20     52.7 MiB      0.0 MiB         999       curr_index = i
    21     52.7 MiB      0.0 MiB      246605       while curr_index > 0 and arr[curr_index-1] > curr_val:
    22     52.7 MiB      0.0 MiB      245606         arr[curr_index] = arr[curr_index-1]
    23     52.7 MiB      0.0 MiB      245606         curr_index -= 1
    24                                         
    25     52.7 MiB      0.0 MiB         999       arr[curr_index] = curr_val




In [38]:
%%writefile merge_sort.py

import random
from memory_profiler import profile

array = []
def init_random_array(array_size):
  global array

  array = []
  random.seed(3102)

  for i in range(array_size):
      array.append(random.randint(0, array_size))
  return (array)

@profile
def merge_sort(array, left_index, right_index):
    if left_index >= right_index:
        return

    middle = (left_index + right_index)//2
    merge_sort(array, left_index, middle)
    merge_sort(array, middle + 1, right_index)
    merge(array, left_index, right_index, middle)

def merge(array, left_index, right_index, middle):
    # Make copies of both arrays we're trying to merge

    # The second parameter is non-inclusive, so we have to increase by 1
    left_copy = array[left_index:middle + 1]
    right_copy = array[middle+1:right_index+1]

    # Initial values for variables that we use to keep
    # track of where we are in each array
    left_copy_index = 0
    right_copy_index = 0
    sorted_index = left_index

    # Go through both copies until we run out of elements in one
    while left_copy_index < len(left_copy) and right_copy_index < len(right_copy):

        # If our left_copy has the smaller element, put it in the sorted
        # part and then move forward in left_copy (by increasing the pointer)
        if left_copy[left_copy_index] <= right_copy[right_copy_index]:
            array[sorted_index] = left_copy[left_copy_index]
            left_copy_index = left_copy_index + 1
        # Opposite from above
        else:
            array[sorted_index] = right_copy[right_copy_index]
            right_copy_index = right_copy_index + 1

        # Regardless of where we got our element from
        # move forward in the sorted part
        sorted_index = sorted_index + 1

    # We ran out of elements either in left_copy or right_copy
    # so we will go through the remaining elements and add them
    while left_copy_index < len(left_copy):
        array[sorted_index] = left_copy[left_copy_index]
        left_copy_index = left_copy_index + 1
        sorted_index = sorted_index + 1

    while right_copy_index < len(right_copy):
        array[sorted_index] = right_copy[right_copy_index]
        right_copy_index = right_copy_index + 1
        sorted_index = sorted_index + 1

# credits: https://stackabuse.com/merge-sort-in-python/

init_random_array(10)
merge_sort(array, 0, len(array)-1)

Writing merge_sort.py


In [39]:
!python merge_sort.py

Filename: c:\Users\Vianiece\Desktop\ICT3113\ICT3113\merge_sort.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    16     52.8 MiB     52.8 MiB           1   @profile
    17                                         def merge_sort(array, left_index, right_index):
    18     52.8 MiB      0.0 MiB           1       if left_index >= right_index:
    19     52.8 MiB      0.0 MiB           1           return
    20                                         
    21                                             middle = (left_index + right_index)//2
    22                                             merge_sort(array, left_index, middle)
    23                                             merge_sort(array, middle + 1, right_index)
    24                                             merge(array, left_index, right_index, middle)


Filename: c:\Users\Vianiece\Desktop\ICT3113\ICT3113\merge_sort.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    16     52.8 MiB     52.8

Notice that the logs are really hard to read. Primarily it is because of the recursive call, hence for the same function gets called recursively many times. Each time it is called it is difficult to see the memory allocated. You may need to run these more than once, obtain the average to get a good stable result.

### 2.4.2 tracemalloc

In [40]:
import tracemalloc

init_random_array(1000)
tracemalloc.start()
start= perf_counter()
insertion_sort(array)
duration = perf_counter() - start
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB; Duration was {duration}")
tracemalloc.stop()


Current memory usage is 0.002449MB; Peak was 0.0206MB; Duration was 0.3129483000084292


In [42]:
import tracemalloc

init_random_array(1000)
tracemalloc.start()
start= perf_counter()
merge_sort(array, 0, len(array)-1)
duration = perf_counter() - start
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB; Duration was {duration}")
tracemalloc.stop()


Current memory usage is 0.367302MB; Peak was 0.385498MB; Duration was 0.011380600015399978


In [43]:
def partition(array, start, end):
    pivot = array[start]
    low = start + 1
    high = end

    while True:
        # If the current value we're looking at is larger than the pivot
        # it's in the right place (right side of pivot) and we can move left,
        # to the next element.
        # We also need to make sure we haven't surpassed the low pointer, since that
        # indicates we have already moved all the elements to their correct side of the pivot
        while low <= high and array[high] >= pivot:
            high = high - 1

        # Opposite process of the one above
        while low <= high and array[low] <= pivot:
            low = low + 1

        # We either found a value for both high and low that is out of order
        # or low is higher than high, in which case we exit the loop
        if low <= high:
            array[low], array[high] = array[high], array[low]
            # The loop continues
        else:
            # We exit out of the loop
            break

    array[start], array[high] = array[high], array[start]

    return high

def quick_sort(array, start, end):
    if start >= end:
        return

    p = partition(array, start, end)
    quick_sort(array, start, p-1)
    quick_sort(array, p+1, end)